In [15]:
import dotenv
import os
import openai
import json
import pandas as pd
import sys
import requests


# .env 파일 로드 (있다면)
dotenv.load_dotenv()

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY") 
SLACK_URL = os.getenv("SLACK_URL")

In [17]:
# 작업 종료 후 slack으로 메신저 보내기기
def send_msg(msg):
    url = SLACK_URL
    message = (msg + " 작업 완료") 
    title = (f"New Incoming Message :zap:") # 타이틀 입력
    slack_data = {
        "username": "NotificationBot", # 보내는 사람 이름
        "icon_emoji": ":satellite:",
        #"channel" : "#somerandomcahnnel",
        "attachments": [
            {
                "color": "#9733EE",
                "fields": [
                    {
                        "title": title,
                        "value": message,
                        "short": "false",
                    }
                ]
            }
        ]
    }
    byte_length = str(sys.getsizeof(slack_data))
    headers = {'Content-Type': "application/json", 'Content-Length': byte_length}
    response = requests.post(url, data=json.dumps(slack_data), headers=headers)
    if response.status_code != 200:
        raise Exception(response.status_code, response.text)

In [12]:
# 데이터 로드
df = pd.read_csv("data_detail_plus.csv").drop(columns = 'text')
df.columns = ['text', 'contents', 'relationships_data', "ERD_data", 'API_specs_data', 'detail']
df_mask = df[['text', 'detail']]
texts = df_mask['text'].to_list()
contents = df_mask['detail'].to_list()

In [30]:
client = openai.OpenAI()

# 최적화된 시스템 프롬프트
OPTIMIZED_SYSTEM_PROMPT = """
당신은 소프트웨어 프로젝트 더미 데이터를 생성하는 전문가입니다.

## 역할과 목표
- 실제 개발 프로젝트에서 사용할 수 있는 현실적이고 구체적인 더미 데이터를 생성합니다
- 다양한 분야(AI, 웹, 모바일, 데이터분석, IoT 등)의 프로젝트를 균형있게 생성합니다
- 최신 기술 트렌드를 반영한 기술 스택을 제안합니다

## 생성 원칙
1. **현실성**: 실제로 구현 가능한 프로젝트여야 합니다
2. **구체성**: 모호하지 않고 명확한 기능과 목표를 가져야 합니다  
3. **다양성**: 서로 다른 분야와 기술 스택을 사용해야 합니다
4. **일관성**: 프로젝트명, 대상, 기능, 기술스택이 서로 논리적으로 연결되어야 합니다
5. **최신성**: 2023-2024년 기준 최신 기술과 트렌드를 반영해야 합니다

## 기술 스택 가이드라인
- **AI/ML**: GPT-4, Claude, Llama, TensorFlow, PyTorch, OpenCV
- **백엔드**: Python, Node.js, Java, Go, Django, FastAPI, Express.js
- **프론트엔드**: React, Vue.js, Angular, Next.js, TypeScript
- **모바일**: React Native, Flutter, Swift, Kotlin
- **데이터베이스**: PostgreSQL, MongoDB, Redis, MySQL, Elasticsearch
- **클라우드**: AWS, Google Cloud, Azure, Docker, Kubernetes

## 응답 형식
- 반드시 JSON 배열 형태로만 응답하세요
- 추가 설명이나 주석 없이 순수한 JSON만 제공하세요
- 모든 텍스트는 한국어로 작성하세요
- 각 필드는 요청된 형식을 정확히 따라야 합니다"""

# 파인튜닝된 모델 ID
MODEL_ID = 'gpt-4o-mini'

def dummy_create(project_overview):
  # 구조화된 요청 프롬프트 (f-string 사용하지 않음)
    enhanced_prompt = f"""
    프로그램 주제 :
    {project_overview}
    
    다음 조건에 맞는 프로젝트 더미 데이터를 JSON 형식으로 아래와 같이 생성해주세요

    ## 생성 조건
    - **프로젝트 이름**: 창의적이고 실제적인 프로젝트명
    - **프로젝트 대상**: 구체적인 타겟 사용자나 업계
    - **메인 기능**: 각 프로젝트마다 3개의 핵심 기능 (AI, 웹, 모바일, 데이터 분석 등 다양한 분야 포함)
    - **기술 스택**: 각 프로젝트마다 3개의 기술 스택 (프로그래밍 언어, 프레임워크, 데이터베이스, AI 모델 등)
    - **프로젝트 설명**: 프로그램 주제와 동일하게 작성해주세요

    ## JSON 형식
    {{
      "projectName": "프로젝트 이름",
      "projectTarget": "프로젝트 대상",
      "mainFunction": [
        "메인 기능1",
        "메인 기능2", 
        "메인 기능3"
      ],
      "techStack": [
        "기술 스택1",
        "기술 스택2",
        "기술 스택3"
      ],
      "projectDescription": "프로젝트 설명"
    }}

    **요청사항**: 
    1. 서로 다른 분야의 프로젝트 **1개만** 생성해주세요
    2. 각 프로젝트는 현실적이고 구현 가능한 수준이어야 합니다
    3. 응답은 JSON 배열 형태로만 제공해주세요 (다른 설명 없이)
    4. 한국어로 작성해주세요

    예시:
    [
      {{
        "projectName": "SmartERD Generator",
        "projectTarget": "소프트웨어 개발자 및 시스템 설계자",
        "mainFunction": [
          "AI 기반 프로젝트 ERD 자동 생성",
          "대시보드에서 실시간 로그 모니터링",
        ],
        "techStack": [
          "Llama 3.1",
          "Python 3.8",
        ],
        "projectDescription": "{project_overview}."
      }}
    ]
    """
    try:
        response = client.chat.completions.create(
            model=MODEL_ID,
            messages=[
                {"role": "system", "content": OPTIMIZED_SYSTEM_PROMPT},
                {"role": "user", "content": enhanced_prompt}
            ],
            max_tokens=4000,
            temperature=0.3
        )
        
        return response.choices[0].message.content
        
    except Exception as e:
        return f"오류 발생: {e}"

In [31]:
text, content = texts[0], contents[0]
result = json.loads(dummy_create(text))
result

[{'projectName': 'StudyBuddy',
  'projectTarget': '대학생 및 직장인 학습자',
  'mainFunction': ['스터디 그룹 생성 및 관리', '퀴즈 생성 및 참여 기능', '공지사항 및 과제 게시판'],
  'techStack': ['Java Servlet', 'JSP', 'MySQL'],
  'projectDescription': 'GQ 서비스는 학습 중심의 스터디 그룹 운영을 지원하는 웹 플랫폼으로, 단순한 그룹 관리에서 나아가 그룹 내 학습 내용을 복습할 수 있는 기능을 제공하여 학습 효과를 극대화하는 것을 목표로 한다. 사용자들은 회원가입과 로그인을 통해 서비스를 이용할 수 있으며, 아이디 및 비밀번호 찾기 기능으로 편의성을 높였다. 사용자는 새로운 스터디 그룹을 생성하거나 기존 그룹을 검색해 가입할 수 있으며, 그룹 내에서는 퀴즈를 생성하고 참여하여 학습한 내용을 효과적으로 반복 학습할 수 있다. 또한 공지사항 게시판을 통해 그룹 내 전달사항을 공유할 수 있고, 과제 게시판은 체크리스트 형태로 구성되어 과제 수행 여부를 명확히 기록할 수 있다. 이러한 기능들은 사용자 간의 소통을 강화하고 학습 동기를 부여하는 데 기여한다. 서비스는 Java Servlet과 JSP 기반으로 구현되었으며, HTML, CSS, JavaScript를 통해 사용자 친화적인 인터페이스를 제공한다. 데이터베이스는 MySQL을 사용하여 안정적인 데이터 저장과 처리를 가능하게 하며, 전반적인 시스템은 학습 관리에 최적화된 구조로 설계되었다.'}]

In [32]:
userdata_lst = []

for text in texts:
    result = json.loads(dummy_create(text))
    userdata_lst.append(result)

df_mask['user_data']= userdata_lst
df_mask.to_csv('data_user_detail_plus.csv', index = 'text')
send_msg('LLM')

C:\Users\mir96\AppData\Local\Temp\ipykernel_8644\682594892.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_mask['user_data']= userdata_lst


In [53]:
df['user_data'] = userdata_lst
df = df.drop(columns= "Unnamed: 0")

In [54]:
df.reset_index().to_csv('data_user_detail_plus.csv', index = False)  

In [57]:
df = pd.read_csv('data_user_detail_plus.csv', index_col= 1)
df.head()

,index,contents,relationships_data,ERD_data,API_specs_data,detail,user_data
text,,,,,,,
"GQ 서비스는 학습 중심의 스터디 그룹 운영을 지원하는 웹 플랫폼으로, 단순한 그룹 관리에서 나아가 그룹 내 학습 내용을 복습할 수 있는 기능을 제공하여 학습 효과를 극대화하는 것을 목표로 한다. 사용자들은 회원가입과 로그인을 통해 서비스를 이용할 수 있으며, 아이디 및 비밀번호 찾기 기능으로 편의성을 높였다. 사용자는 새로운 스터디 그룹을 생성하거나 기존 그룹을 검색해 가입할 수 있으며, 그룹 내에서는 퀴즈를 생성하고 참여하여 학습한 내용을 효과적으로 반복 학습할 수 있다. 또한 공지사항 게시판을 통해 그룹 내 전달사항을 공유할 수 있고, 과제 게시판은 체크리스트 형태로 구성되어 과제 수행 여부를 명확히 기록할 수 있다. 이러한 기능들은 사용자 간의 소통을 강화하고 학습 동기를 부여하는 데 기여한다. 서비스는 Java Servlet과 JSP 기반으로 구현되었으며, HTML, CSS, JavaScript를 통해 사용자 친화적인 인터페이스를 제공한다. 데이터베이스는 MySQL을 사용하여 안정적인 데이터 저장과 처리를 가능하게 하며, 전반적인 시스템은 학습 관리에 최적화된 구조로 설계되었다.",0,"{'project_summary': {'title': 'GQ 서비스', 'categ...","[{'from': 'User', 'to': 'StudyGroup', 'type': ...","[{'name': 'User', 'attributes': [{'name': 'use...","{'api_specification': {'openapi': '3.0.0', 'in...","[{'requirementType': 'FUNCTIONAL1', 'content':...","[{'projectName': 'StudyHub', 'projectTarget': ..."
"‘기억박물관’은 치매 환자와 그 가족을 위한 따뜻한 기억 회복 웹사이트입니다. 사용자는 일상 사진을 업로드하고, 챗봇과의 자연스러운 대화를 통해 사진 속 인물, 장소, 상황에 대한 정보를 되새깁니다. 이렇게 수집된 정보는 자동으로 사용자만의 디지털 다이어리로 정리되며, 환자가 매일 한 번씩 과거의 소중한 기억을 다시 볼 수 있도록 돕는 회상 기능도 포함되어 있습니다. 가족 구성원은 다이어리에 코멘트를 남기거나 설명을 추가할 수 있어, 함께 기억을 공유하고 감정을 나누는 소통의 장이 됩니다. 인지 자극과 정서적 유대감을 동시에 고려한 이 플랫폼은 치매 환자의 삶의 질을 높이고 가족 간의 관계를 더욱 돈독히 만들어주는 기억의 동반자입니다.",1,"{'project_summary': {'title': '기억박물관', 'catego...","[{'from': 'User', 'to': 'Photo', 'type': 'one-...","[{'name': 'User', 'attributes': [{'name': 'use...","{'api_specification': {'openapi': '3.0.0', 'in...","[{'requirementType': 'FUNCTIONAL1', 'content':...","[{'projectName': '기억의 다리', 'projectTarget': '치..."
"이 앱은 보행 약자들을 위한 지하철 길찾기 앱으로, 계단 없이 이동할 수 있는 경로를 안내합니다. 기존의 빠른 환승 정보나 지하철 운행 시간 외에도, 보행 약자를 위한 엘리베이터, 에스컬레이터 위치와 장애물 없는 출입구를 표시해줘 사용자가 편리하게 이동할 수 있습니다. 추가적으로, 실시간 교통 상황, 출입구 혼잡도, 장애물 정보 등을 제공해 안전하고 효율적인 경로를 추천하며, 대중교통 외에도 주변의 장애물 없는 보행 경로를 알려주는 기능도 포함됩니다. 이를 통해 보행약자는 더 많은 독립성을 갖고 편리하게 지하철을 이용할 수 있습니다.",2,{'project_summary': {'title': '보행 약자를 위한 지하철 길...,"[{'from': 'User', 'to': 'Route', 'type': 'one-...","[{'name': 'User', 'attributes': [{'name': 'use...","{'api_specification': {'openapi': '3.0.0', 'in...","[{'requirementType': 'FUNCTIONAL1', 'content':...",[{'projectName': 'EasyAccess Subway Navigator'...
"이 앱은 1인 가구를 위한 맞춤형 플랫폼으로, 주로 커뮤니티 기능에 중점을 두고 있습니다. 사용자는 자신의 주변 100m 이내에 있는 사람들의 글만 볼 수 있어, 동네의 맛집, 추천 가게, 일상 정보 등을 실시간으로 공유하고 소통할 수 있습니다. 또한, 1인 가구의 가장 큰 어려움인 장보기와 배달에 대한 제약을 해결하기 위해 '함께 장보기', '함께 배달하기' 기능을 제공, 여러 사용자가 함께 장보기 목록을 공유하고, 공동 배달 서비스를 통해 비용을 절감할 수 있습니다. 추가로, 긴급한 상황이나 외로움을 느낄 때 사용할 수 있는 ‘이웃과의 도움 요청’ 기능을 통해, 근처에 있는 사람들에게 도움을 청하거나 지원할 수 있어 더 따뜻한 지역사회 형성에 기여합니다. 또한, 1인 가구 전용 할인 혜택이나 쿠폰을 제공하여 경제적 부담을 줄여주는 기능도 탑재되어 있습니다.",3,"{'project_summary': {'title': '1인 가구 맞춤형 플랫폼',...","[{'from': 'User', 'to': 'Post', 'type': 'one-t...","[{'name': 'User', 'attributes': [{'name': 'use...","{'api_specification': {'openapi': '3.0.0', 'in...","[{'requirementType': 'FUNCTIONAL1', 'content':...","[{'projectName': '이웃사촌', 'projectTarget': '1인 ..."
"이 웹앱은 응급실 및 병원 찾기 기능을 제공하여, 응급 상황에서 사용자가 빠르고 안전하게 치료를 받을 수 있도록 돕습니다. 최근 응급실 뺑뺑이 문제를 해결하기 위해, 사용자 위치 기준으로 가장 가까운 응급실을 실시간으로 확인하고, 빈 자리가 있는 응급실을 바로 안내해 줍니다. 또한, 병원에 갈 시간이 맞지 않는 사람들을 위해, 챗봇 기능을 제공해 사용자의 증상에 맞는 질병을 알려주고, 이를 치료할 수 있는 병원과 세부 진료과목(소아과, 이비인후과 등)을 추천해줍니다. 이 앱은 응급 상황에서의 빠르고 정확한 정보 제공을 통해 사용자들이 최적의 의료 서비스를 받는 데 도움을 줍니다.",4,{'project_summary': {'title': '응급실 및 병원 찾기 웹앱'...,"[{'from': 'User', 'to': 'EmergencyRoom', 'type...","[{'name': 'User', 'attributes': [{'name': 'use...","{'api_specification': {'openapi': '3.0.0', 'in...","[{'requirementType': 'FUNCTIONAL1', 'content':...","[{'projectName': 'EmergencyCare Finder', 'proj..."
